# Universidade Federal de Minas Gerais
## Teoria da Decisão - ELE088 
Daniela Amaral Sampaio - 2017074351

Matheus Brito Faria - 2017074386

Victor Emannuel - 2017074394

Importando as bibliotecas que serão utilizadas

In [ ]:
import numpy as np
import pandas as pd
import operator
from sklearn.preprocessing import MinMaxScaler

Fazendo a importação do drive com as tabelas para o trabalho

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Obtendo os dados do arquivo i5x25.xlsx disponível para o trabalho computacional

In [ ]:
path = r'/content/drive/MyDrive/UFMG/9º SEMESTRE/TEORIA DA DECISAO/' \
        'Trabalho-Computacional/i5x25.xlsx'
data = pd.read_excel(path, header=1)
time_job_machine = data[[1, 2, 3, 4, 5]][:25].to_numpy()
cost_late_job = data.iloc[:25, -1].to_numpy()
due_date = data.iloc[-1, 1]

Para esse trabalho,  foi  escolhido  utilizar  o  método ELECTRE  I,  que  constitui  de  uma  problemática  de  escolhaem que se utiliza critério verdadeiro. que trata da problemática de escolha em que se utiliza critério verdadeiro. Nesse método, busca-se encontrar o menor conjunto possível de alternativas não dominadas referentes ao problema em questão. Para isso, são utilizados os índices de concordância e discordância, que medem a vantagem e desvantagem de cada alternativa, respectivamente.

In [ ]:
solutions = [
    [
        [18, 24, 3, 20, 14],
        [23, 22, 0, 9, 5, 19],
        [15, 21, 6, 17, 12],
        [1, 11, 16, 8, 10],
        [2, 13, 7, 4],
    ],
    [
        [11, 24, 18, 14, 20],
        [22, 23, 9, 0, 5, 19],
        [15, 7, 6, 17, 12],
        [1, 21, 8, 16, 10],
        [2, 3, 13, 4],
    ],
    [
        [24, 3, 18, 20, 14],
        [22, 9, 23, 0, 5, 19],
        [8, 15, 7, 6, 17],
        [11, 21, 1, 16, 10],
        [13, 4, 2, 12],
    ],
]

In [ ]:
class CriterionMaker:
    def __init__(self, 
                 schedule=None,
                 time_job_machine=time_job_machine, 
                 cost_late_job=cost_late_job, 
                 due_date=due_date):
        self.time_job_machine = time_job_machine
        self.cost_late_job = cost_late_job
        self.due_date = due_date
        self.number_of_machines = time_job_machine.shape[1]
        self.number_of_jobs = time_job_machine.shape[0]
        self.schedule = schedule
        self.calculate_time_to_finish()
        
    def calculate_time_to_finish(self):
        time_to_finish = np.zeros(self.number_of_jobs)
        for machine, machine_schedule in enumerate(self.schedule):
            late_time = 0
            for job in machine_schedule:
                time_to_finish[job] = self.time_job_machine[job, machine] + late_time
                late_time = time_to_finish[job]
        self.time_to_finish = time_to_finish 

    def calculate_lateness(self):
        return np.maximum(self.time_to_finish-self.due_date, 
                          np.zeros(self.number_of_jobs))

    def calculate_lateness_cost(self):
        return np.sum(self.calculate_lateness()*self.cost_late_job)

    def calculate_advance(self):
        return np.maximum(self.due_date-self.time_to_finish, 
                          np.zeros(self.number_of_jobs))
        
    def calculate_advance_save(self):
        return np.sum(self.calculate_advance()*self.cost_late_job)

    def calculate_makespan(self):
        return np.max(self.time_to_finish)

    def calculate_advance_lateness_tradeoff(self):
        return np.sum((self.due_date-self.time_to_finish)*self.cost_late_job)

    def calculate_sum_job_done(self):
        return np.sum(self.due_date-self.time_to_finish >= 0)

    def run(self):
        return (
            -self.calculate_makespan(),
            -self.calculate_lateness_cost(),
            self.calculate_advance_save(),
            self.calculate_sum_job_done(),
        )


## Critérios adotados pelo decisor 

* Makespan
* Custo de atraso
* "Economia" das tarefas adiantadas
* Quantidades de tarefas realizadas

In [ ]:
criterion_matrix = list()
for solution in solutions:
    criterion_matrix.append(
        CriterionMaker(solution).run()
    )
criterion_matrix = np.array(criterion_matrix) 
criterion_matrix

array([[-11., -73., 290.,  15.],
       [-12., -69., 196.,  14.],
       [-13., -66., 245.,  14.]])

In [ ]:
scaler = MinMaxScaler()
scaler.fit(criterion_matrix)
criterion_matrix = scaler.transform(criterion_matrix)
criterion_matrix

array([[1.        , 0.        , 1.        , 1.        ],
       [0.5       , 0.57142857, 0.        , 0.        ],
       [0.        , 1.        , 0.5212766 , 0.        ]])

Na função ```def calculate_J```, é feita uma comparação par a par entre
todas as alternativas. Em seguida, os índices calculados são comparados com limiares de concordância e de discordância para que sejam estabelecidas as relações de sobreclassificação.



In [ ]:
def calculate_J(criterion_matrix, comparator):
    comparator_dict = {
        '>': operator.gt,
        '<': operator.lt,
        '=': operator.eq,
    }
    comparator = comparator_dict[comparator]
    num_solutions = criterion_matrix.shape[0]
    J = np.empty((num_solutions, num_solutions), dtype=object)
    for i in range(num_solutions):
        for j in range (num_solutions):
            if i == j:
                continue
            J[i, j] = comparator(criterion_matrix[i], criterion_matrix[j])
    return J

In [ ]:
J_plus = calculate_J(criterion_matrix, '>')
J_equal = calculate_J(criterion_matrix, '=')
J_minus = calculate_J(criterion_matrix, '<')

In [ ]:
J_plus

array([[None, array([ True, False,  True,  True]),
        array([ True, False,  True,  True])],
       [array([False,  True, False, False]), None,
        array([ True, False, False, False])],
       [array([False,  True, False, False]),
        array([False,  True,  True, False]), None]], dtype=object)

Foram atribuídos pesos para cada critério usado AHP.

In [ ]:
weights = [0.07811355, 0.3003663 , 0.57802198, 0.04349817]

Convertendo as relações entre ações em valores numéricos na função ```def calculate_P```

In [ ]:
def calculate_P(J, weights):
    num_solutions = J.shape[0]
    P = np.zeros((num_solutions, num_solutions))
    for i in range(num_solutions):
        for j in range(num_solutions):
            if i == j:
                continue
            P[i, j] = np.sum(J[i, j]*weights)
    return P

In [ ]:
P_plus = calculate_P(J_plus, weights)
P_equal = calculate_P(J_equal, weights)
P_minus = calculate_P(J_equal, weights)

In [ ]:
J_plus

array([[None, array([ True, False,  True,  True]),
        array([ True, False,  True,  True])],
       [array([False,  True, False, False]), None,
        array([ True, False, False, False])],
       [array([False,  True, False, False]),
        array([False,  True,  True, False]), None]], dtype=object)

In [ ]:
P_plus

array([[0.        , 0.6996337 , 0.6996337 ],
       [0.3003663 , 0.        , 0.07811355],
       [0.3003663 , 0.87838828, 0.        ]])

Calculando o índice de concordância em ```agreement_index```



In [ ]:
agreement_index = (P_plus + P_equal)/np.sum(weights)
agreement_index

array([[0.        , 0.6996337 , 0.6996337 ],
       [0.3003663 , 0.        , 0.12161172],
       [0.3003663 , 0.92188645, 0.        ]])

Calculando o índice de disconcordância na função ```def calculate_disagreement_index```

In [ ]:
def calculate_disagreement_index(J_minus, criterion_matrix):
    num_solutions = J_minus.shape[0]
    index = np.empty((num_solutions, num_solutions))
    for i in range(num_solutions):
        for j in range(num_solutions):
            if i == j:
                continue
            index[i, j] = np.max(
                criterion_matrix[j]-criterion_matrix[i]
                    ) if np.sum(J_minus[i, j]) else 0
    return index

In [ ]:
disagreement_index = calculate_disagreement_index(J_minus, criterion_matrix)
disagreement_index

array([[0.        , 0.57142857, 1.        ],
       [1.        , 0.        , 0.5212766 ],
       [1.        , 0.5       , 0.        ]])

Fazendo o teste concordância, utilizando como comparação o limiar de concordância de 0.65

In [ ]:
agreement_index > 0.65

array([[False,  True,  True],
       [False, False, False],
       [False,  True, False]])

(1, 2), (1, 3), (3, 2)

Fazendo o teste discordância, utilizando como comparação o limiar de discordância de 0.6

In [ ]:
disagreement_index < 0.6

array([[ True,  True, False],
       [False,  True,  True],
       [False,  True,  True]])

(1, 3), (2, 1), (3, 1)